In [17]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import pickle
import nltk #a python natual language toolkit. Has pruning resources
import pandas as pd
import operator
import unicodedata

import itertools
import numpy as np

from multiprocessing import Pool
from collections import Counter as counter

In [18]:
#Load the charlotte business list that has each business's census tract
char_bizs = pickle.load(open('charlotte_bizs_census.p','rb'))

#Add the zipcode as an element of each business
for business in char_bizs:
    zip_code = business['full_address'][-5:]
    business['zip_code'] = zip_code
char_bizs.sort(key=operator.itemgetter('review_count'),reverse=True)

In [19]:
# Define the census tracts for our samples
gent_cts = [38.05,37,36,41,7,8,9,14,18.02]
ungent_cts = [38.06, 38.07,38.08,38.02,32.01,39.02,39.02,40,43.05,43.03,43.04,43.02,42,45,44,54.01,54.03,54.04,51,50,49,48,46,45,42,52,53.06,23,18.01,17.01,21,19.19,17.02,16.07,16.09,19.12,16.06,16.05,16.03,16.08,15.04,15.09,15.10,13,53.01,52,53.06,53.05,53.06,58.07,54.07,54.01]

In [20]:
revs_g2015 = [] #2015 reviews of businesses in gentrified areas
revs_g2010 = [] #2010 reviews of businesses in gentrified areas
revs_u2015 = [] #2015 reviews of businesses in ungentrified areas
revs_u2010 = [] #2010 reviews of businesses in ungentrified areas

revs_gen_all = [] #all reviews of businesses in gentrified areas, 2010-2015
revs_ungen_all = [] #all reviews of businesses in ungentrified areas, 2010-2015

for biz in char_bizs:
    reviews = biz['reviews']
    ct_num = biz['ct_num'] # census track number
    categoryList = biz['categories']
    
    # Use only businesses that serve food
    if set(categoryList).intersection(['Food','Restaurants','Hotels']):
    
        for rev in reviews:
            year = rev['date'].split('-')[0]
            year = int(year)
            revtext = rev['text']

            # Add review to the appropriate category 
            if (ct_num in gent_cts) and (year == 2015):
                revs_g2015.append(revtext)
            if (ct_num in gent_cts) and (year == 2010):
                revs_g2010.append(revtext)
            if (ct_num in ungent_cts) and (year == 2015):
                revs_u2015.append(revtext)
            if (ct_num in ungent_cts) and (year == 2010):
                revs_u2010.append(revtext)
                
            if (ct_num in gent_cts) and (2010 <= year <= 2015):
                revs_gen_all.append(revtext)
            if (ct_num in ungent_cts) and (2010 <= year <= 2015):
                revs_ungen_all.append(revtext)

In [5]:
print len(revs_g2010), len(revs_g2015), len(revs_u2010),len(revs_u2015)
print len(revs_gen_all), len(revs_ungen_all)

SyntaxError: invalid syntax (<ipython-input-5-f85bff3e456c>, line 1)

In [7]:
num_passes = 20
use_bigrams = True

In [22]:
def create_corpus(reviewList):
    '''A function to produce an lda topic model, 
    given a list of Yelp review texts
    '''

    # I would make these inputs instead of global variable, but I don't know
    # how to make that work with 'map', below
    
    #Check whether or not they're defined yet:
    if (use_bigrams not in locals()) or (num_passes not in locals()):
        print 'Define these in your environment first'

   
    print 'Use Bigrams:'
    print use_bigrams
    print 'Number of Passes:'
    print num_passes
    
    tokenizer = RegexpTokenizer(r'\w+')

    # create English stop words list
    en_stop = get_stop_words('en')

    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    
    # list for tokenized documents in loop
    texts = []

    en_stop.append(u's')
    en_stop.append(u't')
   
    # loop through document list
    for i in reviewList:
    
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

         # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
    
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
        # add tokens to list
        texts.append(stemmed_tokens)
    
    texts_final = []
    
    if use_bigrams:
        for i in texts:
            text_temp = []
            for j in range(len(i) - 1):
                #print (i[j])
                text_temp.append(i[j] + '_' + i[j+1])
            texts_final.append(text_temp)
    else:
        texts_final = texts
    
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts_final)

    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts_final]
    
    return corpus

In [40]:
    
    tokenizer = RegexpTokenizer(r'\w+')

    # create English stop words list
    en_stop = get_stop_words('en')

    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    
    # list for tokenized documents in loop
    texts = []

    en_stop.append(u's')
    en_stop.append(u't')
   
    # loop through document list
    for i in revs_gen_all:#revs_ungen_all
    
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

         # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
    
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
        # add tokens to list
        texts.append(stemmed_tokens)
    
    texts_final = []
    
    for i in texts:
        for j in range(len(i) - 1):
            #print (i[j])
            texts_final.append(i[j] + '_' + i[j+1])
        #texts_final.append(text_temp)
    texts_final = counter(texts_final)
   
    dictWords = texts_final.most_common()
    texts_final = pd.DataFrame(dictWords)
    texts_final.to_csv('WordCount_gent.csv',encoding='utf-8')

In [23]:
def char_ldamodel(reviewList):
    
    corpus = create_corpus(reviewList)

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=num_passes)

    return ldamodel

In [10]:
def output_topic_model(ldamodel,output_fname='',num_words=10):
    ''' For outputting the results of a topic model 
    into a more human-readable format
    '''
    
    #Format final output
    mat = ldamodel.print_topics(num_topics=10, num_words=num_words)
    final_output = []
    
    for i in mat:
        topic = i[1]
        weights_words = topic.split('+')
        words = [w.split('*')[1] for w in weights_words] #Get rid of weights
        words = [unicodedata.normalize('NFKD', word).encode('ascii','ignore') for word in words]
        final_output.append(words)
    
    output_frame = pd.DataFrame(final_output)

    # If a filename has been specified, save the topic model
    if output_fname:
        output_frame.to_csv(output_fname,encoding='utf-8')
   
    return output_frame


In [11]:
# We have four topic models to run; let's run them all at once to save time
def easy_parallize(f, sequence):

    from multiprocessing import Pool
    pool = Pool(processes=4) # number of cores(threads?)
    
    result = pool.map(f, sequence) #a list of all the topic models requested
    
    # Dispose of any empty results (needed only if #processes != len(sequence)?)
    cleaned = [x for x in result if not x is None] 
    cleaned = np.asarray(cleaned)
   
    # not optimal but safe
    pool.close()
    pool.join()
    
    return cleaned

In [ ]:
# produce the four topic models, one for each list of reviews
#%time results = easy_parallize(char_ldamodel,[revs_g2015,revs_g2010,revs_u2015,revs_u2010])

In [14]:
#Let's try not using bigrams
#%time monoresults = easy_parallize(char_ldamodel,[revs_g2015,revs_g2010,revs_u2015,revs_u2010])

CPU times: user 207 ms, sys: 101 ms, total: 308 ms
Wall time: 3min 49s


In [92]:
#Let's try using many passes, and bigrams
# mp stands for 'many passes'
#%time bigram_mp = easy_parallize(char_ldamodel,[revs_g2015,revs_g2010,revs_u2015,revs_u2010])

CPU times: user 7.35 s, sys: 2.45 s, total: 9.8 s
Wall time: 2h 1min 38s


In [95]:
#Let's try using many passes, and unigrams
#%time unigram_mp = easy_parallize(char_ldamodel,[revs_g2015,revs_g2010,revs_u2015,revs_u2010])

CPU times: user 6.49 s, sys: 2.08 s, total: 8.57 s
Wall time: 2h 27min 18s


In [ ]:
#Let's using many passes, and unigrams
#%time unigram_mp = easy_parallize(char_ldamodel,[revs_g2015,revs_g2010,revs_u2015,revs_u2010])

In [ ]:
# Let's generate a topic model for all the 2015 data, and all the 2010 data

In [19]:

# Save or load the topic model results

#pickle.dump(results,open('tms_bigrams.p','wb'))
#pickle.dump(monoresults,open('tms_monograms.p','wb'))
#pickle.dump(bigram_mp,open('tms_bigrams_mp.p','wb'))
#pickle.dump(unigram_mp,open('tms_unigrams_mp.p','wb'))

bigram_mods = pickle.load(open('tms_bigrams_mp.p','rb'))
unigram_mods = pickle.load(open('tms_unigrams_mp.p','rb'))

# Save the results

mod_g2015 = bigram_mods[0] # unigram topic model for 2015, gentrified-area reviews
mod_g2010 = bigram_mods[1]
mod_u2015 = bigram_mods[2]
mod_u2010 = bigram_mods[3]

mod_g2015u = unigram_mods[0] # unigram topic model for 2015, gentrified-area reviews
mod_g2010u = unigram_mods[1]
mod_u2015u = unigram_mods[2]
mod_u2010u = unigram_mods[3]

#Output results 

num_words = 10
g2015 = output_topic_model(mod_g2015,'tm_mp_bigram_10w_gent_2015.csv',num_words)
g2010 = output_topic_model(mod_g2010,'tm_mp_bigram_10w_gent_2010.csv',num_words)
u2015 = output_topic_model(mod_u2015,'tm__mp_bigram_10w_ungent_2015.csv',num_words)
u2010 = output_topic_model(mod_u2010,'tm_mp_bigram_10w_ungent_2010.csv',num_words)
g2015u = output_topic_model(mod_g2015u,'tm_mp_unigram_10w_gent_2015.csv',num_words)
g2010u = output_topic_model(mod_g2010u,'tm_mp_unigram_10w_gent_2010.csv',num_words)
u2015u = output_topic_model(mod_u2015u,'tm_mp_unigram_10w_ungent_2015.csv',num_words)
u2010u = output_topic_model(mod_u2010u,'tm_mp_unigram_10w_ungent_2010.csv',num_words)


**The above calls were from when this notebook was used to both make topic models and examine them. There will be some issues with the variable names if you try to run all the cells above**

In [33]:
# Create and output both unigram and bigram topic models,
# using all the reviews from 2010-2015, 
# for gentrified an ungentrified regions

num_passes = 1000
use_bigrams = True
%time bigram_all = easy_parallize(char_ldamodel,[revs_gen_all,revs_ungen_all])
pickle.dump(bigram_all,open('tms_all_bigrams_mp.p','wb'))
#We write out the entire collection of topic models

num_passes = 1000
use_bigrams = False #passing so-called inputs to functions like this is bad form, but 
%time unigram_all = easy_parallize(char_ldamodel,[revs_gen_all,revs_ungen_all])
pickle.dump(bigram_all,open('tms_all_unigram_mp.p','wb'))

mod_gent = bigram_all[0] # bigram topic model for gentrified-area reviews 2010-2015
mod_ungent = bigram_all[1] # bigram topic model for gentrified-area reviews 2010-2015
g_all = output_topic_model(mod_gent,'tm_all_mp_bigram_10w_gent.csv',num_words)
ung_all = output_topic_model(mod_ungent,'tm_all_mp_bigram_10w_ungent.csv',num_words)


mod_gent = unigram_all[0] # unigram topic model for gentrified-area reviews 2010-2015
mod_ungent = unigram_all[1] # unigram topic model for gentrified-area reviews 2010-2015
g_all_u = output_topic_model(mod_gent,'tm_all_mp_unigram_10w_gent.csv',num_words)
ung_all_u = output_topic_model(mod_ungent,'tm_all_mp_unigram_10w_ungent.csv',num_words)


CPU times: user 5.4 s, sys: 1.5 s, total: 6.9 s
Wall time: 2h 45min 45s
CPU times: user 5.22 s, sys: 1.3 s, total: 6.51 s
Wall time: 2h 42min 22s


In [12]:
print use_bigrams

True


In [13]:
bigram_all = easy_parallize(char_ldamodel,[revs_gen_all,revs_ungen_all])

Define these in your environment first
Define these in your environment first
Use Bigrams:
Use Bigrams:
True
True
Number of Passes:
Number of Passes:
20
20


In [16]:
pickle.dump(bigram_all,open('tms_all_bigrams_mp.p','wb'))

In [18]:
num_words = 10
mod_gent = bigram_all[0] # bigram topic model for gentrified-area reviews 2010-2015
mod_ungent = bigram_all[1] # bigram topic model for gentrified-area reviews 2010-2015
g_all = output_topic_model(mod_gent,'tm_all_mp_bigram_10w_gent.csv',num_words)
ung_all = output_topic_model(mod_ungent,'tm_all_mp_bigram_10w_ungent.csv',num_words)

In [7]:
allmodel = create_corpus(revs_gen_all + revs_ungen_all)

NameError: name 'create_corpus' is not defined

In [24]:
num_words = 30
allmodel_readable = output_topic_model(mod_gent,'tm_all_GenAndUngen_bigram_10w_gent.csv',num_words)

In [25]:
allmodel_readable

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,food_truck,pale_ale,tap_room,dog_friendli,go_back,tripl_c,room_clean,live_music,park_lot,fish_taco,...,can_wait,tast_room,can_find,outdoor_seat,first_impress,realli_good,good_thing,oliv_garden,pretti_good,even_though
1,mac_chees,grill_chees,sweet_potato,hyatt_place,beer_select,great_food,go_back,reason_price,great_beer,corn_hole,...,look_forward,will_definit,good_food,come_back,will_never,first_time,park_lot,pimento_chees,can_say,great_place
2,fri_chicken,realli_good,picnic_tabl,pretti_good,good_food,go_back,come_back,fish_taco,food_truck,brown_ale,...,stay_hyatt,absolut_love,long_time,pretti_much,pretti_decent,great_food,call_order,end_get,look_forward,fresh_veggi
3,go_back,crepe_cellar,pretti_good,n_chees,mac_n,come_back,pimento_chees,tast_like,mac_chees,next_time,...,wal_mart,first_time,great_place,food_good,fri_chicken,goat_chees,one_best,will_never,front_desk,fish_taco
4,front_desk,free_rang,custom_servic,fri_chicken,go_back,beer_select,5_star,hyatt_hous,friendli_staff,craft_beer,...,larg_portion,come_back,friend_order,littl_bit,never_stay,coffe_shop,haven_tri,can_wait,grab_beer,salt_caramel
5,go_back,pull_pork,fri_chicken,realli_like,realli_good,come_back,first_time,saturday_morn,watch_game,drive_thru,...,didn_see,staff_friendli,definit_go,crab_cake,reason_price,pretti_good,next_time,realli_enjoy,friendli_staff,give_tri
6,great_place,pork_belli,food_truck,fish_taco,park_lot,go_back,realli_good,come_back,de_mayo,first_time,...,even_though,just_open,daili_press,next_time,south_end,coupl_time,outdoor_seat,good_beer,coffe_drink,much_food
7,fish_taco,food_good,ve_ever,beer_select,divin_pie,realli_good,look_like,great_place,come_back,go_back,...,small_batch,pot_pie,steak_hous,peopl_watch,sweet_potato,next_time,4_star,saturday_night,sport_bar,great_peopl
8,go_back,fri_chicken,ice_cream,come_back,park_lot,first_time,ten_minut,beer_select,tast_like,next_time,...,custom_servic,give_tri,bleu_barn,last_time,good_food,pleasantli_surpris,bbq_taco,barn_bistro,look_like,don_think
9,food_truck,truck_friday,la_quinta,tasti_yo,smelli_cat,great_beer,great_place,place_stay,can_wait,farmer_market,...,everi_time,realli_great,way_get,frozen_yogurt,ve_ever,ve_seen,nice_staff,can_bring,allow_dog,one_favorit
